In [21]:
import os
import getpass
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [22]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

client = OpenAI()

In [14]:
persist_directory = './persist'

# Load the Chroma vector store
chroma_vs = Chroma(persist_directory=persist_directory, embedding_function=OpenAIEmbeddings())

/tmp/ipykernel_7417/3789767084.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_vs = Chroma(persist_directory=persist_directory, embedding_function=OpenAIEmbeddings())


In [18]:
query = "What's the maternity leave policy?"
What's the maternity leave policy?
results = chroma_vs.similarity_search(query=query)

for result in results:
    print(result.page_content+"\n")

TechLance’s parental leave policies integrate closely with our childcare support programs. New mothers receive 12 weeks of paid maternity leave, while non-birth parents receive six weeks of paid paternity leave. Adoptive parents receive eight weeks of paid leave that can be shared between both parents. Employees must have been with the company for at least 12 months to qualify for paid parental leave, though unpaid leave options may be available for newer employees under FMLA guidelines.During

TechLance’s parental leave policies integrate closely with our childcare support programs. New mothers receive 12 weeks of paid maternity leave, while non-birth parents receive six weeks of paid paternity leave. Adoptive parents receive eight weeks of paid leave that can be shared between both parents. Employees must have been with the company for at least 12 months to qualify for paid parental leave, though unpaid leave options may be available for newer employees under FMLA guidelines.During



In [39]:
def get_response(messages):
    """
    Generate a response using the chat completions API
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

In [40]:
def append_message(messages, role, content):
    """Append a message to the conversation history."""
    messages.append({"role": role, "content": content})
    return messages

In [51]:
# Obtain answer from RAG
rag_results = chroma_vs.similarity_search(query=query)
rag_result = rag_results[0].page_content

# Initialize conversation
messages = [
    {"role": "system", "content": "You are an assistant summarizing policies into employee-friendly explanations."},
    {"role": "user", "content": rag_result}
]

response = get_response(messages)
messages = append_message(messages, "assistant", response)

In [52]:
response

"At TechLance, we understand the importance of family, which is why we offer supportive parental leave policies that complement our childcare programs. Here's what you need to know:\n\n- **Maternity Leave**: New mothers can take **12 weeks of paid leave** after the birth of their child.\n- **Paternity Leave**: Non-birth parents (like partners or spouses) are eligible for **6 weeks of paid leave**.\n- **Adoptive Parents**: Those who adopt can receive **8 weeks of paid leave**, which can be shared between both parents.\n\nTo qualify for paid parental leave, employees need to have been with TechLance for **at least 12 months**. If you've been here for less time, don't worry—there may be unpaid leave options available for you under FMLA guidelines.\n\nWe’re here to support you during this exciting time in your life!"

In [45]:
rag_result

'TechLance’s parental leave policies integrate closely with our childcare support programs. New mothers receive 12 weeks of paid maternity leave, while non-birth parents receive six weeks of paid paternity leave. Adoptive parents receive eight weeks of paid leave that can be shared between both parents. Employees must have been with the company for at least 12 months to qualify for paid parental leave, though unpaid leave options may be available for newer employees under FMLA guidelines.During'

In [46]:
query_2 = "What if I am adopting a child?"
messages = append_message(messages, "user", query_2)
get_response(messages)

'If you are adopting a child at TechLance, you are eligible for 8 weeks of paid leave. This leave can be shared between both parents if applicable. Just remember that to qualify for this benefit, you need to have been with the company for at least 12 months. If you have any questions or need assistance with the adoption leave process, feel free to reach out to HR!'

In [48]:
rag_results_aux = chroma_vs.similarity_search(query=query_2)
rag_results_aux[0].page_content

'For international adoptions, additional unpaid leave may be available for required travel, and we oﬀer limited reimbursement for adoption-related travel expenses.'

In [53]:
query_2 = "What if I am adopting a child?"
query_2 += "\n" + rag_results_aux[0].page_content


messages = append_message(messages, "user", query_2)
get_response(messages)

"If you're adopting a child, you'll be entitled to **8 weeks of paid leave** that can be shared between both parents. \n\nIf you're pursuing an international adoption, you may also have the option for additional **unpaid leave** to cover any necessary travel. Plus, we offer **limited reimbursement** for eligible adoption-related travel expenses to help ease the financial burden.\n\nWe’re committed to supporting you throughout the adoption process!"

In [ ]:
# Initialize conversation
messages = [
    {"role": "system", "content": "You are an assistant summarizing policies into employee-friendly explanations."}
]
while True:
    # Let the user ask a question
    query = input()

    # Get the most similar answer from the RAG
    rag_results = chroma_vs.similarity_search(query=query)
    rag_result = rag_results[0].page_content # Get the most similar one

    # Concatenate the user query with the rag response
    final_query = query + "\n" + rag_result

    # Append user message
    messages = append_message(messages, "user", final_query)

    # Get response from the chatbot and add it to the message to continue the context
    response = get_response(messages)
    messages = append_message(messages, "assistant", response)
    print(response+"\n")

 What's the maternity leave policy


At TechLance, we support our employees during their journey into parenthood with the following maternity leave benefits:

- **Maternity Leave**: New mothers get **12 weeks of paid leave**.
- **Paternity Leave**: Non-birth parents can take **6 weeks of paid leave**.
- **Adoption Leave**: For adoptive parents, there’s **8 weeks of paid leave**, which can be shared between both parents.

To be eligible for paid parental leave, you need to have been with TechLance for at least **12 months**. If you’ve been here for less than a year, you may still have options for unpaid leave under FMLA guidelines.



 What if you are adopting a kid?


If you're adopting a child, TechLance offers the following benefits:

- **Adoption Leave**: You will receive **8 weeks of paid leave** that can be shared between both parents, whether you are adopting alone or with a partner.
  
- **International Adoptions**: If you are adopting from another country, you may qualify for **additional unpaid leave** for travel purposes.

- **Travel Expense Reimbursement**: We also provide limited reimbursement for expenses related to adoption travel.

This ensures you have the necessary time and financial support during your adoption process.

